### Importing data sets
    will be using MNIST data set imported from pytorch. 
    In the preprocessing stages
### TO-DO
    how to properly work with images (normally used to numbers)
    how to preprocess this data

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch 
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt



Importing data sets from pytorch. and storing them within the correct variables

In [2]:

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Taking a look at the data. How many Data Samples do we have? 60000

Size of each data? 28x28. Grayscale was already applied

In [9]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [10]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [11]:
train_data.targets.size()

torch.Size([60000])

In [12]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

Creating loaders. Used for loading in data. Batch size of about 100 images at a time. This is where each image will be stored for the model we will be using. 

In [37]:


loaders = {

    'train': DataLoader(train_data,
                        batch_size = 100,
                        shuffle=True,
                        num_workers=1),
    
    'test': DataLoader(train_data,
                        batch_size = 100,
                        shuffle=True,
                        num_workers=1)
    
}


In [15]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f12b3e8ce60>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f12b4098e00>}

Creating class structure for model we will be using. Convolutional Neural network otherwise known as a MLP (Multilayer perceptron)

In [41]:

class CNN(nn.Module):

    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
            # hidden layer 1
        self.conv2 = nn.Conv2d(10, 2, kernel_size = 5)  
            # hidden layer 2
        self.conv2_drop = nn.Dropout2d()
        # dropout layer will randomly drop out or deactivate certain neurons during training
        # does not change shape of data
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)

    def forward(self, x): #manually creating the neurons for CNN
        x = F.relu(F.max_pool2d(self.conv1(x),2)) # max pooling function
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2)) 
        # want to reshape and flatten linear data 
        x = x.view(-1, 320) 
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)

        return F.softmax(x)

Optimization and Training Code block. (More manual since we are using pytorch)

In [46]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
    # checking to see if we can use GPU 

model = CNN().to(device) 
    # move model to correct architecture

optimizer = optim.Adam(model.parameters(), lr=0.001) 
    # optimizer with learning rate (0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        print(f'Target size: {target.shape}')
        print(f'Output size: {output.shape}')
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100 * batch_idx/ len(loaders["train"]):.0f}%)]\t{loss.item():.6}')

def test():
    model.eval()
    
    test_loss  = 0
    correct = 0

    with torch.no_grad():
        for data, targets in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct+=pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct/ len(loaders["test"].dataset):.0f}%\n)')

            

In [45]:
for epoch in range(1, 11):
    train(epoch)
    test()

Target size: tensor([3, 1, 6, 2, 6, 4, 4, 2, 8, 2, 7, 0, 8, 4, 9, 8, 2, 1, 8, 1, 0, 5, 0, 4,
        9, 3, 1, 0, 1, 2, 6, 5, 3, 6, 5, 6, 8, 4, 8, 9, 1, 6, 8, 2, 9, 8, 8, 8,
        0, 2, 6, 0, 6, 0, 5, 7, 0, 9, 1, 1, 6, 1, 3, 7, 0, 0, 7, 1, 3, 0, 6, 7,
        4, 2, 5, 9, 1, 4, 4, 5, 6, 0, 7, 1, 0, 8, 5, 5, 5, 2, 0, 9, 6, 8, 0, 2,
        8, 0, 3, 2])
Output size: tensor([[0.0088, 0.0110, 0.0099, 0.0112, 0.0108, 0.0119, 0.0095, 0.0086, 0.0099,
         0.0084, 0.0089, 0.0110, 0.0091, 0.0108, 0.0099, 0.0103, 0.0097, 0.0104,
         0.0108, 0.0084, 0.0107, 0.0108, 0.0106, 0.0099, 0.0095, 0.0110, 0.0095,
         0.0089, 0.0099, 0.0105, 0.0113, 0.0104, 0.0099, 0.0109, 0.0107, 0.0098,
         0.0088, 0.0096, 0.0111, 0.0102, 0.0098, 0.0084, 0.0106, 0.0105, 0.0097,
         0.0090, 0.0111, 0.0093, 0.0098, 0.0105, 0.0115, 0.0088, 0.0110, 0.0113,
         0.0088, 0.0114, 0.0108, 0.0101, 0.0095, 0.0107, 0.0098, 0.0094, 0.0109,
         0.0104, 0.0087, 0.0093, 0.0095, 0.0090, 0.0101, 0.0109, 0

/tmp/ipykernel_87505/1583925582.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


ValueError: Expected input batch_size (10) to match target batch_size (100).

In [7]:
device

device(type='cpu')

In [8]:
model.eval()

data, target = test_data[0]

data = data.unsqueeze(0).to(device)

output = model(data)

prediction = output.argmax(dim=1, keepdim=True).item

print(f'Prediciton: {prediction}')

image = data.squeeze(0).squeeze(0).cpu().numpy()

plt.imshow(image, cmap = 'gray')
plt.show()

RuntimeError: shape '[-1, 320]' is invalid for input of size 32